<a href="https://colab.research.google.com/github/fabiobanyu/File_Tugas/blob/main/Tugas_Damin_Pertemuan5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

# --- Load dataset ---
file_path = "/content/house_prices.csv"
df = pd.read_csv(file_path, engine='python')

# Preprocessing kolom nominal
df["Ownership"] = df["Ownership"].astype("category")
df["Transaction"] = df["Transaction"].astype("category")

# Preprocessing kolom numerik
df["Carpet Area"] = df["Carpet Area"].str.replace(" sqft", "", regex=False)
df["Carpet Area"] = pd.to_numeric(df["Carpet Area"], errors="coerce")

# Buang NaN hanya di kolom yang dipakai analisis
df_clean = df.dropna(subset=["Ownership", "Transaction", "Price (in rupees)", "Carpet Area"])

print("Jumlah data sebelum cleaning:", len(df))
print("Jumlah data setelah cleaning:", len(df_clean))


df_clean.head()

Jumlah data sebelum cleaning: 83079
Jumlah data setelah cleaning: 28915


,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
1,1,2 BHK Ready to Occupy Flat for sale in Dosti V...,One can find this stunning 2 BHK flat for sale...,98 Lac,13799.0,thane,473.0,Ready to Move,3 out of 22,Resale,...,East,Garden/Park,Dosti Vihar,2,NaN,1 Open,Freehold,NaN,NaN,NaN
2,2,2 BHK Ready to Occupy Flat for sale in Sunrise...,Up for immediate sale is a 2 BHK apartment in ...,1.40 Cr,17500.0,thane,779.0,Ready to Move,10 out of 29,Resale,...,East,Garden/Park,Sunrise by Kalpataru,2,NaN,1 Covered,Freehold,NaN,NaN,NaN
4,4,2 BHK Ready to Occupy Flat for sale in TenX Ha...,"This lovely 2 BHK Flat in Pokhran Road, Thane ...",1.60 Cr,18824.0,thane,635.0,Ready to Move,20 out of 42,Resale,...,West,"Garden/Park, Main Road",TenX Habitat Raymond Realty,2,NaN,1 Covered,Co-operative Society,NaN,NaN,NaN
9,9,3 BHK Ready to Occupy Flat for sale in Pride P...,One can find this stunning 3 BHK flat for sale...,1.60 Cr,11150.0,thane,900.0,Ready to Move,3 out of 27,Resale,...,East,Garden/Park,Pride Palms,3,1,1 Covered,Freehold,NaN,NaN,NaN
10,10,3 BHK Ready to Occupy Flat for sale in Cosmos ...,Up for immediate sale is a 3 BHK apartment in ...,1.40 Cr,12174.0,thane,950.0,Ready to Move,6 out of 20,Resale,...,NaN,Main Road,Cosmos Lounge,2,NaN,1 Open,Co-operative Society,NaN,NaN,NaN


#**PERTEMUAN 4**

In [2]:
# Ambil 200 sampel untuk variabel nominal
sample_nominal = df_clean[["Ownership", "Transaction"]].sample(200, random_state=42)

# Buat tabel kontingensi
contingency_table = pd.crosstab(sample_nominal["Ownership"], sample_nominal["Transaction"])

# Uji Chi-Square
chi2, p_val, dof, expected = chi2_contingency(contingency_table)

print("=== Chi-Square Test (Ownership vs Transaction) ===")
print("Chi-Square Value:", chi2)
print("p-value:", p_val)
print("Degrees of Freedom:", dof)
print("\nContingency Table:")
print(contingency_table)

=== Chi-Square Test (Ownership vs Transaction) ===
Chi-Square Value: 3.091388044579534
p-value: 0.3777484369548202
Degrees of Freedom: 3

Contingency Table:
Transaction           New Property  Resale
Ownership                                 
Co-operative Society             4       3
Freehold                        70     118
Leasehold                        0       3
Power Of Attorney                1       1


In [3]:
numerical_cols = ["Price (in rupees)", "Carpet Area"]
sample_numeric = df_clean[numerical_cols].head(10)

# Hitung kovarians & korelasi
cov_matrix = sample_numeric.cov()
corr_matrix = sample_numeric.corr()

print("\n=== Sample Numerical Data (10 rows) ===")
print(sample_numeric)

print("\n=== Covariance Matrix ===")
print(cov_matrix)

print("\n=== Correlation Matrix ===")
print(corr_matrix)


=== Sample Numerical Data (10 rows) ===
    Price (in rupees)  Carpet Area
1             13799.0        473.0
2             17500.0        779.0
4             18824.0        635.0
9             11150.0        900.0
10            12174.0        950.0
13            17526.0       1820.0
15            10000.0        675.0
16             5736.0        647.0
18            11250.0        600.0
20             8571.0        430.0

=== Covariance Matrix ===
                   Price (in rupees)    Carpet Area
Price (in rupees)       1.807078e+07  714936.888889
Carpet Area             7.149369e+05  158066.766667

=== Correlation Matrix ===
                   Price (in rupees)  Carpet Area
Price (in rupees)           1.000000     0.423018
Carpet Area                 0.423018     1.000000


#**PERTEMUAN 5**

In [14]:
import math

#Memilih 10 Data Unik Langsung dari DataFrame ---
print("Memilih 10 data unik pertama dari kolom 'Price (in rupees)'...")

# Mengambil semua nilai unik dari kolom harga
harga_unik = df_clean['Price (in rupees)'].unique()

# Mengambil 10 baris pertama dari nilai-nilai unik tersebut
harga_asli = pd.Series(harga_unik[:10])

print("Data yang terpilih:\n", harga_asli.to_list(), "\n")
print("-" * 30)

#Min-Max Normalization
min_val = harga_asli.min()
max_val = harga_asli.max()
min_max_normalized = (harga_asli - min_val) / (max_val - min_val)

#Z-Score Normalization
mean_val = harga_asli.mean()
# ddof=0 digunakan untuk menghitung standar deviasi populasi
std_val = harga_asli.std(ddof=0)
z_score_normalized = (harga_asli - mean_val) / std_val

#Decimal Scaling Normalization
max_abs_val = harga_asli.abs().max()
k = math.ceil(math.log10(max_abs_val))
decimal_scaling_normalized = harga_asli / (10**k)

#Membandingkan hasil normalisasi
comparison_df = pd.DataFrame({
    'Harga Asli': harga_asli,
    'Normalisasi Min-Max': min_max_normalized,
    'Normalisasi Z-Score': z_score_normalized,
    'Normalisasi Decimal Scaling': decimal_scaling_normalized
})

# Mengatur format angka pada tabel agar mudah dibaca
comparison_df['Harga Asli'] = comparison_df['Harga Asli'].map('{:,.0f}'.format)
pd.options.display.float_format = '{:,.4f}'.format

# --- Membuat Tabel Gabungan ---

# Membuat DataFrame tunggal dari semua hasil
final_table = pd.DataFrame({
    'Harga Asli': harga_asli,
    'Normalisasi Min-Max': min_max_normalized,
    'Normalisasi Z-Score': z_score_normalized,
    'Normalisasi Decimal Scaling': decimal_scaling_normalized
})

print(final_table.to_markdown(index=False))


Memilih 10 data unik pertama dari kolom 'Price (in rupees)'...
Data yang terpilih:
 [13799.0, 17500.0, 18824.0, 11150.0, 12174.0, 17526.0, 10000.0, 5736.0, 11250.0, 8571.0] 

------------------------------
|   Harga Asli |   Normalisasi Min-Max |   Normalisasi Z-Score |   Normalisasi Decimal Scaling |
|-------------:|----------------------:|----------------------:|------------------------------:|
|        13799 |              0.616061 |              0.284168 |                       0.13799 |
|        17500 |              0.898839 |              1.20189  |                       0.175   |
|        18824 |              1        |              1.53019  |                       0.18824 |
|        11150 |              0.413661 |             -0.372691 |                       0.1115  |
|        12174 |              0.491901 |             -0.118775 |                       0.12174 |
|        17526 |              0.900825 |              1.20833  |                       0.17526 |
|        10000 |  